In [ ]:
# Poner semillas para demostrar mejora?

# ResNet-34
#Duda: No entiendo como se supone que se utiliza el stride = (2,2), pues hacer esto cambia las dimensiones del tensor.
# ¿Como esperan luego que sume los dos tensores de dimension diferente? Para arreglarlo, lo único que se me ocurrió
# (asumo está mal) es que, en caso de que haya stride=(2,2) ,la suma se realice entre el resultado de la primera convolusion que altera las dimensiones, 
# y lo que ocurra luego de hacer la segunda convolusión. 

# El input original para  ResNet-34 es de (224,224,3), mientras que el de Cifar-10 es de (32,32,3). ¿Qué hacemos para arreglarlo?

In [10]:
import tensorflow as tf
 
# Display the version
print(tf.__version__)    

# other imports
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, Add, ReLU
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.models import Model

2.12.0


In [6]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10
 
# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [7]:
# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
 
# flatten the label values
y_train, y_test = y_train.flatten(), y_test.flatten()

In [ ]:
'''
Opción 1:
tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=avg,
    classes=10,
    classifier_activation='softmax'
)

https://keras.io/api/applications/resnet/
'''

In [43]:
def identity_block(inp, filters: int):
    # Corresponde a los bloque normales, donde solo hay stripe=(1,1)

    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1), padding="same", kernel_initializer='he_normal')(inp)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1), padding="same")(inp)
    x = BatchNormalization()(x)
    x = Add()([inp,x])
    x = Activation('relu')(x)
    return x

def conv_block(inp, filters: int):
    # Corresponde al primer bloque de cada grupo, el cual tiene stripe=(2,2)
    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(2,2), padding="same")(inp)
    x = BatchNormalization()(x)
    shortcut = Activation('relu')(x)

    x = Conv2D(filters=filters, kernel_size=(3,3), strides=(1,1), padding="same")(shortcut)
    x = BatchNormalization()(x)
    x = Add()([shortcut,x])
    x = Activation('relu')(x)
    return x

In [44]:
classes = len(set(y_train))

i = Input(shape=x_train[0].shape)
x = Conv2D(filters=64, kernel_size=(7,7), strides=(2,2), padding='same')(i) # Originalmente stride=(7,7)
x = Activation('relu')(x)
x = MaxPooling2D((3,3), strides=(2,2))(x)

x = identity_block(x, 64)
x = identity_block(x, 64)
x = identity_block(x, 64)

x = conv_block(x, 128)
x = identity_block(x, 128)
x = identity_block(x, 128)
x = identity_block(x, 128)


x = conv_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)
x = identity_block(x, 256)

x = conv_block(x, 512)
x = identity_block(x, 512)
x = identity_block(x, 512)

x = AveragePooling2D((1,1))(x) # Alterar esta para encajar en nuestro input
x = Dense(classes, activation='softmax')(x) # Originalmente es 1000, pero para cifar10 dee ser 10

model = Model(i, x)
 
# model description
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_456 (Conv2D)            (None, 16, 16, 64)   9472        ['input_21[0][0]']               
                                                                                                  
 activation_451 (Activation)    (None, 16, 16, 64)   0           ['conv2d_456[0][0]']             
                                                                                                  
 max_pooling2d_19 (MaxPooling2D  (None, 7, 7, 64)    0           ['activation_451[0][0]']         
 )                                                                                          

In [39]:
# Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [40]:
# Fit
r = model.fit(
  x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Epoch 1/20
  13/1563 [..............................] - ETA: 14:55 - loss: 4.3187 - accuracy: 0.1029

KeyboardInterrupt: 

In [ ]:
plt.plot(r.history['accuracy'], label='acc', color='red')
plt.plot(r.history['val_accuracy'], label='val_acc', color='green')
plt.legend()

In [ ]:
# https://github.com/ke511081177/Resnet34_keras/blob/main/resnet34.py 
# Resnet34 de alguien más, que no termino de comprender.

# https://stackoverflow.com/questions/64792460/how-to-code-a-residual-block-using-two-layers-of-a-basic-cnn-algorithm-built-wit
# Bloque residual